# Нейроэволюционный алгоритм

## Загружаем и подготавливаем данные для работы

### Строки, не несущие информацию были удалены

In [1]:
import datatable as dt
data = dt.fread('cancer1.dt')
del data[0:7,:]

## Преробразуем данные в формат pandas.DataFrame 

In [2]:
import pandas as pd
df = pd.DataFrame(data.to_pandas())
df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
0,0.2,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1,True,False
1,0.2,0.1,0.1,0.1,0.2,0.1,0.3,0.1,0.1,True,False
2,0.5,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1,True,False
3,0.5,0.4,0.6,0.8,0.4,0.1,0.8,1.0,0.1,False,True
4,0.5,0.3,0.3,0.1,0.2,0.1,0.2,0.1,0.1,True,False
...,...,...,...,...,...,...,...,...,...,...,...
694,0.5,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1,True,False
695,0.7,0.5,0.6,1.0,0.5,1.0,0.7,0.9,0.4,False,True
696,0.6,1.0,1.0,1.0,0.8,1.0,0.7,1.0,0.7,False,True
697,0.5,0.7,1.0,1.0,0.5,1.0,1.0,1.0,0.1,False,True


## Разделяем данные на входные и требуемые выходные значения

In [3]:
outputs = df[['C9','C10']] \
                    .iloc[:,0]\
                    .astype('int64')

outputs

0      1
1      1
2      1
3      0
4      1
      ..
694    1
695    0
696    0
697    0
698    1
Name: C9, Length: 699, dtype: int64

In [4]:
inputs = df.iloc[:, 0:9].copy().astype('float64')
inputs

,C0,C1,C2,C3,C4,C5,C6,C7,C8
0,0.2,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1
1,0.2,0.1,0.1,0.1,0.2,0.1,0.3,0.1,0.1
2,0.5,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1
3,0.5,0.4,0.6,0.8,0.4,0.1,0.8,1.0,0.1
4,0.5,0.3,0.3,0.1,0.2,0.1,0.2,0.1,0.1
...,...,...,...,...,...,...,...,...,...
694,0.5,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
695,0.7,0.5,0.6,1.0,0.5,1.0,0.7,0.9,0.4
696,0.6,1.0,1.0,1.0,0.8,1.0,0.7,1.0,0.7
697,0.5,0.7,1.0,1.0,0.5,1.0,1.0,1.0,0.1


## Разбиваем выборку на тренировочную и тестовую

In [5]:
from sklearn.model_selection import train_test_split

inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, train_size = 0.8, random_state = 3)

In [6]:
inputs_train.info()
inputs_test.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 108 to 664
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   C0      559 non-null    float64
 1   C1      559 non-null    float64
 2   C2      559 non-null    float64
 3   C3      559 non-null    float64
 4   C4      559 non-null    float64
 5   C5      559 non-null    float64
 6   C6      559 non-null    float64
 7   C7      559 non-null    float64
 8   C8      559 non-null    float64
dtypes: float64(9)
memory usage: 43.7 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 386 to 326
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   C0      140 non-null    float64
 1   C1      140 non-null    float64
 2   C2      140 non-null    float64
 3   C3      140 non-null    float64
 4   C4      140 non-null    float64
 5   C5      140 non-null    float64
 6   C6      140 non-null    float64
 7   C7

In [7]:
from NeurlaNetwork import NeuralNetwork
import settings
import numpy as np

$BLX-{\alpha}$ кроссинговер.

In [14]:
def crossbreeding(nn_1, nn_2, BLX_A):
  nn_of_1 = NeuralNetwork(shape=nn_1.shape)
  nn_of_2 = NeuralNetwork(shape=nn_2.shape)

  for i in range(len(nn_1.weights)):
    for j in range(len(nn_1.weights[i])):
      for k in range(len(nn_1.weights[i][j])):
        c_min = np.minimum(nn_1.weights[i][j][k],nn_2.weights[i][j][k])
        c_max = np.maximum(nn_1.weights[i][j][k],nn_2.weights[i][j][k])
        delat_k = c_max-c_min
        nn_of_1.weights[i][j][k] = np.random.uniform(c_min-delat_k*BLX_A,c_max-delat_k*BLX_A)
        nn_of_2.weights[i][j][k] = np.random.uniform(c_min-delat_k*BLX_A,c_max-delat_k*BLX_A)

  return nn_of_1, nn_of_2

Функция для прямого распространения в ансамбле ИНС.

In [9]:
def forward_prop_ensemble(ensemble,X):
  for count in range(len(ensemble)):
      ensemble[count].forward_propagation(X)

"Голосование" ансамбля ИНС (результат вероятность).

In [10]:
def ensemble_bagging_preds(ensemble):
  F = 0
  for count in range(len(ensemble)):
    F += np.sum(ensemble[count].neurons[-1])/len(ensemble)
  return F

"Голосование" ансамбля ИНС (дискретный результат - 0 или 1).

In [11]:
def ensemble_bagging(ensemble):
  F = 0
  for count in range(len(ensemble)):
    F += np.sum(ensemble[count].neurons[-1])/len(ensemble)
  if F>0.5:
    return 1
  else:
    return 0

Функция для оценивания точности обученного классификатора на тестовой выборке.

In [12]:
def ensemble_est(ensemble,X,y):
  # размер тестового датасета
  data_set_size = y.shape[0]
  # количество правильно классифицированных случаев
  correct = 0
  for i in range(data_set_size):
    for count in range(len(ensemble)):
      population[count].forward_propagation(X.iloc[i,:].to_numpy())
    prediction = ensemble_bagging(ensemble)
    if prediction == y.iloc[i]:
      correct +=1
  accuracy = correct/data_set_size*100
  print(f'accuracy: {accuracy}%')
  return accuracy

Основной алгоритм

In [15]:

# формирование начальной популяции
population = []
model_accuracy_array = []
epochs_array = []

# инициализация начальной популяции
for count in range(settings.NNS_IN_POPULATION_COUNTS):
  neural_network = NeuralNetwork(shape=settings.SHAPE_NN)
  population.append(neural_network)

error_f = np.zeros(len(population))

for epoch in range(settings.EPOCHS):

  sort_indx = error_f.argsort() # индексы отсортированных элементов

  # получаем двух особей усечением (лучшая половина популяции)
  individual_1 = 0
  individual_2 = 0
  while individual_1 == individual_2:
    individual_1 = np.random.randint(0, settings.NNS_IN_POPULATION_COUNTS / 2)
    individual_2 = np.random.randint(0, settings.NNS_IN_POPULATION_COUNTS / 2)

  # Кроссинговер BLX-a. Получаем двух потомков от двух родителей
  nn_of_1,nn_of_2=crossbreeding(population[sort_indx[individual_1]],population[sort_indx[individual_2]],settings.BLX_A)

# Удаляем особи с наибольшим значением функции ошибки
  if sort_indx[-1]>sort_indx[-2]:
    population.remove(population[sort_indx[-1]])
    population.remove(population[sort_indx[-2]])
  else:
    population.remove(population[sort_indx[-2]])
    population.remove(population[sort_indx[-1]])

  population.append(nn_of_1)
  population.append(nn_of_2)

  # Гауссовская мутация
  for mut_count in range(settings.MUTATION_COUNTS):
    mut = np.random.randint(0,len(population))
    population[mut].mutation()
  # Обнулим значения в массиве значений функции ошибок
  error_f[:]=0
  # Цикл обучения
  for learning_cycle in range(settings.LEARNING_SAMPLES):
    #выбираем случайный индекс элемент тренировочного датасета
    df_index = np.random.randint(0,inputs_train.shape[0])
    # входные данные
    piece_of_data = inputs_train.iloc[df_index,:].to_numpy()
    # желаемый отклик
    d = outputs_train.iloc[df_index]
    p = np.zeros_like(error_f)
    # прямое распространение для всей популяции
    forward_prop_ensemble(population,piece_of_data)

    # общий вывод популяции
    F = ensemble_bagging_preds(population)

    # расчёт функции корреляционного штрафа
    for count in range(len(population)):
      for count_p in range(len(population)):
        if count_p != count:
          p[count] += (population[count].neurons[-1]-F)*(population[count_p].neurons[-1]-F)
      error_f[count]+=1/settings.LEARNING_SAMPLES/2*(population[count].neurons[-1]-d)**2+1/settings.LEARNING_SAMPLES*settings.CORRELATION_CONSTANT*p[count]
  if epoch%50==0:
    epochs_array.append(epoch)
    print(f'epoch: {epoch}')
    model_accuracy_array.append(ensemble_est(population,inputs_test,outputs_test))


AttributeError: 'NeuralNetwork' object has no attribute 'w'

Далее оценим точность классификатора на тестовой выборке.

In [ ]:
ensemble_est(population,inputs_test,outputs_test)

График обучения

In [ ]:
import matplotlib.pyplot as plt

plt.plot(epochs_array, model_accuracy_array, c='blue')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

Демонстрация вероятностного вывода.

In [ ]:
df_index = np.random.randint(0,outputs_test.shape[0])
piece_of_data = outputs_test.iloc[df_index,:].to_numpy()
d = outputs_test.iloc[df_index]
for count in range(len(population)):
  population[count].forward_propagation(piece_of_data)
F = ensemble_bagging_preds(population)
print(d)
print(F)

Функция для вывода весов связей в отдельный файл.

In [ ]:
def save_weights(ensemble):
  file = open('NN_ensemble_structure.txt','w')
  for nn in range(len(ensemble)):
    file.write('\n########################\n')
    for i in range(len(ensemble[nn].w)):
      for j in range(len(ensemble[nn].w[i])):
        for k in range(len(ensemble[nn].w[i][j])):
          file.write(str(ensemble[nn].w[i][j][k])+' ')
        file.write('\n')
  file.close()


In [ ]:
save_weights(population)

Функция для записи весов смещения.

In [ ]:
def save_bias(ensemble):
  file = open('NN_ensemble_bias.txt','w')
  for nn in range(len(ensemble)):
    file.write('\n########################\n')
    for i in range(len(ensemble[nn].b)):
      for j in range(len(ensemble[nn].b[i])):
        file.write(str(ensemble[nn].b[i][j])+' ')
      file.write('\n')
  file.close()

In [ ]:
save_bias(population)